In [45]:
# import libraries needed
from pathlib import Path
import os, yaml, time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# load in data
file_name = '../../Data/CubicBox_(-250.0, 250.0)_(-250.0, 250.0)_(-250.0, 250.0).npz'
particle_subsample = np.load(file_name)
x, y, z = particle_subsample['x'], particle_subsample['y'], particle_subsample['z']
vx, vy, vz = particle_subsample['vx'], particle_subsample['vy'], particle_subsample['vz'] 
x_range,y_range,z_range=[min(x),max(x)],[min(y),max(y)],[min(z),max(z)]

In [46]:
# split the data into the correct boxes
def full_density_histogram(grid,x=x,y=y,z=z):
    combined=np.column_stack((x,y,z))
    hist, edges = np.histogramdd(combined, bins=[grid,grid,grid])
    return hist, edges

In [47]:
# produce just a simple density plot
def density_plot(z1,grid,x_range=x_range,y_range=y_range):
    bins=int(500/grid)
    hist,edges=full_density_histogram(bins)
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slicey=hist[:,:,z_bin[0]]
    plt.imshow(slicey, extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    plt.colorbar(label='Number of particles')
    path = '../Figures/Density plot along z=({:.0f},{:.0f}) (gridsize of {:.0f} mpc cubed).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

In [48]:
# produce a histogram of the densities in a particular slice
def histogram2d(z1,grid):
    bins=int(500/grid)
    hist,edges=full_density_histogram(bins)
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slicey=hist[:,:,z_bin[0]]
    slicey=slicey.flatten()
    plt.hist(np.log10(slicey+1),bins=25)
    plt.xlabel('Log10 density')
    plt.ylabel('Frequency')
    path = '../Figures/Histogram of densities along z=({:.0f},{:.0f}) (gridsize of {:.0f} mpc cubed).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

In [49]:
# produce just a simple overdensity plot
def overdensity_plot(z1,grid,x_range=x_range,y_range=y_range,x=x):
    fullhist,ed=full_density_histogram(int(500/grid))
    smolboxvol=grid**3
    densities=fullhist/smolboxvol
    otherav=len(x)/(500**3)
    overdense=densities/otherav
    z_bin = np.where((ed[2][:-1] >= z1) & (ed[2][:-1] < z1+grid))[0]
    slices=overdense[:,:,z_bin[0]]
    plt.imshow(np.log10(slices), extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.colorbar(label='$\log_{10}(\delta+1)$')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    path = '../Figures/Overdensities along z=({:.0f},{:.0f}) (gridsize of {:.0f} mpc cubed).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

<>:11: SyntaxWarning: invalid escape sequence '\l'
<>:11: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_59159/2930326021.py:11: SyntaxWarning: invalid escape sequence '\l'
  plt.colorbar(label='$\log_{10}(\delta+1)$')


In [50]:
# produce density plot with velocities overlayed
def density_with_velocities(z1,grid,x_range=x_range,y_range=y_range,vy=vy,vx=vx,x=x,y=y,z=z):
    bins=int(500/grid)
    hist,edges=full_density_histogram(bins)
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slicey=hist[:,:,z_bin[0]]
    velocitymask=(z<z1+grid)&(z>=z1)
    posx=x[velocitymask]
    posy=y[velocitymask]
    xvelo=vx[velocitymask]
    yvelo=vy[velocitymask]
    x_indices=np.digitize(posx,edges[0])-1
    y_indices=np.digitize(posy,edges[1])-1
    xidx = np.clip(x_indices, 0, bins-1)
    yidx = np.clip(y_indices, 0, bins-1)
    directx=np.zeros((bins,bins))
    directy=np.zeros((bins,bins))
    counts=np.zeros((bins,bins))
    np.add.at(directx, (xidx, yidx), xvelo)
    np.add.at(directy, (xidx, yidx), yvelo)
    np.add.at(counts, (xidx, yidx), 1)
    directx[counts > 0] /= counts[counts > 0]
    directy[counts > 0] /= counts[counts > 0]
    checkerx=directx.T.flatten()
    checkery=directy.T.flatten()
    oo=np.array(list(np.linspace(x_range[0]+5,x_range[1]-5,bins))*bins)
    oo=np.sort(oo)
    ee=np.array(list(np.linspace(y_range[0]+5,y_range[1]-5,bins))*bins)
    plt.quiver(oo,ee,checkerx,checkery,scale_units='xy',angles='xy', color='r',label='x-y velocities (km/s)')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    plt.legend(loc=2)
    plt.imshow(slicey, extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.colorbar(label='Number of particles')
    path = '../Figures/Velocity density plot along z=({:.0f},{:.0f}) (gridsize of {:.0f} mpc cubed).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return    

In [51]:
# produce overdensity plot with velocities overlayed
def overdensity_with_velocities(z1,grid,x_range=x_range,y_range=y_range,vy=vy,vx=vx,x=x,y=y,z=z):
    smolboxvol=grid**3
    bins=int(500/grid)
    hist,edges=full_density_histogram(bins)
    densities=hist/smolboxvol
    otherav=len(x)/(500**3)
    overdense=densities/otherav
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slices=overdense[:,:,z_bin[0]]
    velocitymask=(z<z1+grid)&(z>=z1)
    posx=x[velocitymask]
    posy=y[velocitymask]
    xvelo=vx[velocitymask]
    yvelo=vy[velocitymask]
    x_indices=np.digitize(posx,edges[0])-1
    y_indices=np.digitize(posy,edges[1])-1
    xidx = np.clip(x_indices, 0, bins-1)
    yidx = np.clip(y_indices, 0, bins-1)
    directx=np.zeros((bins,bins))
    directy=np.zeros((bins,bins))
    counts=np.zeros((bins,bins))
    np.add.at(directx, (xidx, yidx), xvelo)
    np.add.at(directy, (xidx, yidx), yvelo)
    np.add.at(counts, (xidx, yidx), 1)
    directx[counts > 0] /= counts[counts > 0]
    directy[counts > 0] /= counts[counts > 0]
    checkerx=directx.T.flatten()
    checkery=directy.T.flatten()
    oo=np.array(list(np.linspace(x_range[0]+5,x_range[1]-5,bins))*bins)
    oo=np.sort(oo)
    ee=np.array(list(np.linspace(y_range[0]+5,y_range[1]-5,bins))*bins)
    plt.imshow(np.log10(slices), extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.colorbar(label='$\log_{10}(\delta+1)$')
    plt.quiver(oo,ee,checkerx,checkery,scale_units='xy',angles='xy', color='r',label='x-y velocities (km/s)')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    plt.legend(loc=2)
    path = '../Figures/Velocity density plot along z=({:.0f},{:.0f}) (gridsize of {:.0f} mpc cubed).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return    

<>:34: SyntaxWarning: invalid escape sequence '\l'
<>:34: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_59159/1414678961.py:34: SyntaxWarning: invalid escape sequence '\l'
  plt.colorbar(label='$\log_{10}(\delta+1)$')


In [52]:
# calculate the mean and std of each box
# function can be varied to return component means and stds
def meanvelo_std(grid,x=x,y=y,z=z):
    bins=int(500/grid)
    hist,edges=full_density_histogram(bins)
    x_indices=np.digitize(x,edges[0])-1
    y_indices=np.digitize(y,edges[1])-1
    z_indices=np.digitize(z,edges[1])-1
    xidx = np.clip(x_indices, 0, bins-1)
    yidx = np.clip(y_indices, 0, bins-1)
    zidx = np.clip(z_indices, 0, bins-1)
    directx=np.zeros((bins,bins,bins))
    directy=np.zeros((bins,bins,bins))
    directz=np.zeros((bins,bins,bins))
    stdxsq=np.zeros((bins,bins,bins))
    stdysq=np.zeros((bins,bins,bins))
    stdzsq=np.zeros((bins,bins,bins))
    stdx=np.zeros((bins,bins,bins))
    stdy=np.zeros((bins,bins,bins))
    stdz=np.zeros((bins,bins,bins))
    counts=np.zeros((bins,bins,bins))
    np.add.at(directx, (xidx, yidx, zidx), vx)
    np.add.at(directy, (xidx, yidx, zidx), vy)
    np.add.at(directz, (xidx, yidx, zidx), vz)
    np.add.at(counts, (xidx, yidx, zidx), 1)
    directx[counts > 0] /= counts[counts > 0]
    directy[counts > 0] /= counts[counts > 0]
    directz[counts > 0] /= counts[counts > 0]
    np.add.at(stdxsq, (xidx, yidx, zidx), (vx-directx[xidx,yidx,zidx])**2)
    np.add.at(stdysq, (xidx, yidx, zidx), (vy-directx[xidx,yidx,zidx])**2)
    np.add.at(stdzsq, (xidx, yidx, zidx), (vz-directx[xidx,yidx,zidx])**2)
    stdx[counts > 0] = np.sqrt(stdxsq[counts > 0] / counts[counts>0])
    stdy[counts > 0] = np.sqrt(stdysq[counts > 0] / counts[counts>0])
    stdz[counts > 0] = np.sqrt(stdzsq[counts > 0] / counts[counts>0])
    stdt=np.sqrt((stdx**2+stdy**2+stdz**2)/3) # technically an rms
    means=np.sqrt(directx**2+directy**2+directz**2)
    return means, stdt

In [53]:
# produce histogram of all velocity and std data
def produce_velo_std_histogram(grid):
    means,stdt=meanvelo_std(grid)
    plt.hist(means.flatten(),bins=25)
    plt.xlabel('Mean velocity (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/Histogram of all velocities (gridsize of {:.0f} mpc cubed).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    plt.hist(stdt.flatten(),bins=25)
    plt.xlabel('Velocity $\sigma$ (km/s)')
    plt.ylabel('Frequency')
    path='../Figures/Histogram of standard deviations in all velocities (gridsize of {:.0f} mpc cubed).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_59159/4283607464.py:11: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('Velocity $\sigma$ (km/s)')


In [54]:
# choose gridsize and slice to be investigated
gridsize=10
zlim=150

In [55]:
# calling each of the functions
histogram2d(zlim,gridsize)
density_plot(zlim,gridsize)
overdensity_plot(zlim,gridsize)
density_with_velocities(zlim,gridsize)
overdensity_with_velocities(zlim,gridsize)
produce_velo_std_histogram(gridsize)

In [ ]:
# THE FOLLOWING CODE ISN'T GREAT AND THUS ISN'T IN USE

In [ ]:
# NOT IN USE
# Plotting velocities
vx_dist, edgesvx = np.histogram(vx, bins=20)
vy_dist, edgesvy = np.histogram(vy, bins=20)
vz_dist, edgesvz = np.histogram(vz, bins=20)
centresvx = edgesvx[:-1] + (edgesvx[1] - edgesvx[0])/2
centresvy = edgesvy[:-1] + (edgesvy[1] - edgesvy[0])/2
centresvz = edgesvz[:-1] + (edgesvz[1] - edgesvz[0])/2
widthvx = (edgesvx[1] - edgesvx[0]) * 0.95
widthvy = (edgesvy[1] - edgesvy[0]) * 0.95
widthvz = (edgesvz[1] - edgesvz[0]) * 0.95
plot_heightvx = max(np.log10(vx_dist))+0.1*max(np.log10(vx_dist))
plot_heightvy = max(np.log10(vy_dist))+0.1*max(np.log10(vy_dist))
plot_heightvz = max(np.log10(vz_dist))+0.1*max(np.log10(vz_dist))
fig, (axs1, axs2, axs3) = plt.subplots(1, 3, sharex=False, sharey=False, figsize=(18,6))
axs1.bar(centresvx, np.log10(vx_dist+1), width=widthvx, color='r', alpha=0.8, label='vx-coord distribution')
axs2.bar(centresvy, np.log10(vy_dist+1), width=widthvy, color='g', alpha=0.8, label='vy-coord distribution')
axs3.bar(centresvz, np.log10(vz_dist+1), width=widthvz, color='b', alpha=0.8, label='vz-coord distribution')
axs1.set_ylim([0, plot_heightvx])
axs2.set_ylim([0, plot_heightvy])
axs3.set_ylim([0, plot_heightvz])
axs1.legend(loc="upper left")
axs1.set_xlabel('vx (km/s)')
axs1.set_ylabel('log10 particle count')
axs2.legend(loc="upper left")
axs2.set_xlabel('vy (km/s)')
axs2.set_ylabel('log10 particle count')
axs3.legend(loc="upper left")
axs3.set_xlabel('vz (km/s)')
axs3.set_ylabel('log10 particle count')
path = '../Figures/Particle velocity distribution along the axes.png'
plt.savefig(path, dpi=300)
plt.close()  

In [ ]:
# NOT IN USE
def create_mask(x_start, y_start, z_start, size, x=x, y=y, z=z):
    newmask=(x>=x_start)&(x<x_start+size)&(y>=y_start)&(y<y_start+size)&(z>=z_start)&(z<z_start+size)
    newx=x[newmask]
    newy=y[newmask]
    newz=z[newmask]
    newvx=vx[newmask]
    newvy=vy[newmask]
    newvz=vz[newmask]
    return newx,newy,newz,newvx,newvy,newvz

In [ ]:
# NOT IN USE
def create_2d_density_plot(x_data,y_data,x_start,y_start,z_start,size,bins):
    histdata, xed, yed, colours = plt.hist2d(x_data,y_data,bins=bins)
    cbar = plt.colorbar(colours)
    cbar.set_label('Density')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    path = '../Figures/Density plot of size {:.0e} mpc cubed from x=({:.0f},{:.0f}), y=({:.0f},{:.0f}), z=({:.0f},{:.0f}).png'.format(size**3,x_start,x_start+size,y_start,y_start+size,z_start,z_start+size)
    plt.savefig(path, dpi=300)
    plt.close()
    return histdata

In [ ]:
# NOT IN USE
def create_3d_vector_plot(x_data,y_data,z_data,vx_data,vy_data,vz_data,x_start,y_start,z_start,size,sample):
    ax=plt.figure().add_subplot(111,projection='3d')
    ax.scatter(x_data[::sample],y_data[::sample],z_data[::sample],color='k')
    ax.quiver(x_data[::sample],y_data[::sample],z_data[::sample],vx_data[::sample],vy_data[::sample],vz_data[::sample],length=0.002,color='r',alpha=0.5)
    ax.set_xlabel('x coords')
    ax.set_ylabel('y coords')
    ax.set_zlabel('z coords')
    path = '../Figures/Sample of every {:.0f} velocity vectors from x=({:.0f},{:.0f}), y=({:.0f},{:.0f}), z=({:.0f},{:.0f}) for box {}.png'.format(sample, x_start,x_start+size,y_start,y_start+size,z_start,z_start+size,box_ID)
    plt.savefig(path, dpi=300)
    plt.close()
    return

In [ ]:
# NOT IN USE
def create_histogram(histdata,bins,x_start,y_start,z_start,size):
    plt.hist(np.log10(histdata+1),bins=bins)
    plt.xlabel('Log10 density')
    plt.ylabel('Frequency')
    path = '../Figures/Histogram of density plot from x=({:.0f},{:.0f}), y=({:.0f},{:.0f}), z=({:.0f},{:.0f}).png'.format(x_start,x_start+size,y_start,y_start+size,z_start,z_start+size)
    plt.savefig(path, dpi=300)
    plt.close()
    return